In [30]:
import pandas as pd

In [31]:
secop2023 = pd.read_csv('../../muestras de datos/bronce/SECOP_2023.csv', sep=';')
secop2023.head(2)

C:\Users\Asus\AppData\Local\Temp\ipykernel_11512\2229726427.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  secop2023 = pd.read_csv('../../muestras de datos/bronce/SECOP_2023.csv', sep=';')


,ID_ENTIDAD,index,ID_Contrato,ID_Proceso,Modalidad,Estado,Descripcion_proceso,Objeto_Contrato,Tipo_de_contrato,Fecha_firma,...,Fecha_fin_contrato,Link,Fuente,Año_firma,Modalidad Estandar,Grupo Modalidad,SEGMENTO_UNSPSC,FAMILIA_UNSPSC,CLASE_UNSPSC,PRODUCTO_UNSPSC
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Otras Modalidades de Contratación,Otra Modalidad,NaN,NaN,NaN,NaN
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,Otras Modalidades de Contratación,Otra Modalidad,NaN,NaN,NaN,NaN


In [32]:
nits = pd.read_excel('../../muestras de datos/auxiliar/criterios/educacion/Listado_NIT_Ent_Edu.xlsx')
nits.head(2)

,NIT,Entidad,Categoría
0,860.024.301-6,ICFES,Ministerio de educación
1,830.018.957-3,FODESEP,Ministerio de educación


In [33]:
# limpieza de nits
nits['NIT'] = nits['NIT'].str.replace('.', '')
nits['NIT'] = nits['NIT'].str.split('-').str[0]
nits['NIT'] = nits['NIT'].str.split(':').str[0]
nits = nits[~(nits['NIT'].isna())]
nits.head(2)

,NIT,Entidad,Categoría
0,860024301,ICFES,Ministerio de educación
1,830018957,FODESEP,Ministerio de educación


In [34]:
homologa_entidades = pd.read_excel('../../muestras de datos/auxiliar/homologacion de entidades.xlsx')
homologa_entidades = homologa_entidades.drop_duplicates(subset=homologa_entidades.columns)
homologa_entidades['NIT'] = homologa_entidades['NIT'].astype(str)
# Cargue para determinar los id que se deberán tener en cuenta
cruce_homologa_entidades = pd.merge(left=nits, right=homologa_entidades, how='left', on='NIT')
cruce_homologa_entidades.shape

(426, 9)

In [36]:
cruce_homologa_entidades = homologa_entidades[homologa_entidades['ID'].isin(cruce_homologa_entidades['ID'])]
cruce_homologa_entidades.shape

(439, 7)

In [37]:
cruce_homologa_entidades.columns

Index(['ID', 'ID_SECOP', 'ENTIDAD', 'NIT', 'ORDEN', 'DEPARTAMENTO',
       'MUNICIPIO'],
      dtype='object')

In [38]:
secop2023_filtrado = secop2023[secop2023['ID_ENTIDAD'].isin(cruce_homologa_entidades['ID'])] 
secop2023_filtrado = secop2023_filtrado[~(secop2023_filtrado['ID_Contrato'].isna())]
secop2023_filtrado.shape

(30898, 28)

In [39]:
secop2023_filtrado['Fuente'].value_counts()

Fuente
SECOP I     25342
SECOP II     5556
Name: count, dtype: int64

In [10]:
# secop2023_filtrado.to_excel('../../muestras de datos/plata/educacion_2023.xlsx', index=False)

In [40]:
secop2023_bronce = pd.read_csv('../../../../OneDrive - Colombia Compra Eficiente/Preparacion_Datos_Abiertos/muestras de datos/procesados/bronce/SECOP_2023.csv', sep=';')
secop2023_bronce.shape

C:\Users\Asus\AppData\Local\Temp\ipykernel_11512\815130178.py:1: DtypeWarning: Columns (1,5,13,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  secop2023_bronce = pd.read_csv('../../../../OneDrive - Colombia Compra Eficiente/Preparacion_Datos_Abiertos/muestras de datos/procesados/bronce/SECOP_2023.csv', sep=';')


(1307189, 27)

In [41]:
columnas_entidad = secop2023_bronce[['ID_Contrato','ID_Entidad', 'Nombre_Entidad', 'NIT_Entidad', 'Orden_Entidad']]
columnas_entidad['ID_Contrato'] = columnas_entidad['ID_Contrato'].astype(str)

C:\Users\Asus\AppData\Local\Temp\ipykernel_11512\2000933314.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  columnas_entidad['ID_Contrato'] = columnas_entidad['ID_Contrato'].astype(str)


In [42]:
secop2023_filtrado2 = pd.merge(left=secop2023_filtrado, right=columnas_entidad, how='inner', on='ID_Contrato', indicator=True)
secop2023_filtrado2 = secop2023_filtrado2.drop_duplicates(subset=secop2023_filtrado.columns)
secop2023_filtrado2.shape

(30898, 33)

In [44]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words("spanish"))

# Función Limpieza de Texto
def remove_extra_punct(text):
    
    text = text.lower()
    text = re.sub(r'(?::|;|=)(?:-)?(?:\)|\(|D|P)', "", text)
    text = re.sub(r'[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]', "", text)
    text = re.sub(r'\#\.', '', text)
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'  ', ' ', text)
    text = re.sub(r'´', '',text)
    text = re.sub(r',', '',text)
    text = re.sub(r'\-', '', text)
    text = re.sub(r'á', 'a', text)
    text = re.sub(r'é', 'e', text)
    text = re.sub(r'í', 'i', text)
    text = re.sub(r'ó', 'o', text)
    text = re.sub(r'ú', 'u', text)
    text = re.sub(r'ò', 'o', text)
    text = re.sub(r'à', 'a', text)
    text = re.sub(r'è', 'e', text)
    text = re.sub(r'ì', 'i', text)
    text = re.sub(r'ù', 'u', text)
    text = re.sub("\d+", ' ', text)
    text = re.sub("\\s+", ' ', text)
    
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if not w.lower() in stop_words]
    tokens = " ".join(tokens)
    
    return tokens

secop2023_filtrado2['Nombre_Entidad'] = secop2023_filtrado2['Nombre_Entidad'].apply(remove_extra_punct)
secop2023_filtrado2 = secop2023_filtrado2.drop(columns='_merge')

In [45]:
secop2023_filtrado3 = pd.merge(left=secop2023_filtrado2, right=homologa_entidades[['ENTIDAD', 'DEPARTAMENTO', 'MUNICIPIO']], how='left',left_on='Nombre_Entidad', right_on='ENTIDAD', indicator=True)
print(secop2023_filtrado3.shape)
secop2023_filtrado3['_merge'].value_counts()

(31003, 36)


_merge
both          30757
left_only       246
right_only        0
Name: count, dtype: int64

In [47]:
secop2023_filtrado3 = secop2023_filtrado3.drop(columns='index')
secop2023_filtrado3.to_excel('../../muestras de datos/plata/educacion_secopi_secopii.xlsx', index=False)

TVEC

In [4]:
tvec = pd.read_csv('../../muestras de datos/bronce/TVEC2023_DA.csv')
tvec.head(2)

C:\Users\Asus\AppData\Local\Temp\ipykernel_11512\3151868813.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  tvec = pd.read_csv('../../muestras de datos/bronce/TVEC2023_DA.csv')


,Año,Identificador de la Orden,Agregacion,Rama de la Entidad,Sector de la Entidad,Entidad,Orden de la Entidad,NIT Entidad,Solicitante,Fecha,Proveedor,Estado,Solicitud,Items,Total,Ciudad,Entidad Obigada,EsPostconflicto,NIT proveedor,Actividad Economica Proveedor
0,2023,104914,Combustible Nacional,No Definido,No Definido,No Definido,No Definido,NaN,Juan Luna Toro,02/17/2023 12:00:00 AM,Sodexo SAS,Emitida,157390.0,Combustible Categora C,3183000.0,Cali,No Definido,No Definido,800219876,8299
1,2023,104914,Combustible Nacional,No Definido,No Definido,No Definido,No Definido,NaN,Juan Luna Toro,02/17/2023 12:00:00 AM,Sodexo SAS,Emitida,157390.0,Combustible Categora C,3183000.0,Cali,No Definido,No Definido,800219876,8299


In [8]:
cruce_homologa_entidades.head()

,NIT,Entidad,Categoría,ID,ID_SECOP,ENTIDAD,ORDEN,DEPARTAMENTO,MUNICIPIO
0,860024301,ICFES,Ministerio de educación,1935.0,122002002.0,instituto colombiano fomento educacion superio...,NACIONAL,Bogotá D.C.,Bogotá D.C.
1,860024301,ICFES,Ministerio de educación,1935.0,702412826.0,instituto colombiano evaluacion educacion icfes,Nacional,Distrito Capital de Bogotá,No Definido
2,830018957,FODESEP,Ministerio de educación,NaN,NaN,NaN,NaN,NaN,NaN
3,899999336,AMAZONAS,Secretarías de educación,111.0,291000001.0,amazonas gobernacion,TERRITORIAL,Amazonas,Leticia
4,890900286,ANTIOQUIA,Secretarías de educación,264.0,205000001.0,antioquia gobernacion,TERRITORIAL,Antioquia,Medellín


In [9]:
len(cruce_homologa_entidades['NIT'].unique())

145

In [11]:
nits_homologados = homologa_entidades[homologa_entidades['ID'].isin(cruce_homologa_entidades['ID'])]
print(nits_homologados.shape)
nits_homologados.head(2)

(439, 7)


,ID,ID_SECOP,ENTIDAD,NIT,ORDEN,DEPARTAMENTO,MUNICIPIO
5,2,205000052,antioquia universidad antioquia,890980040,TERRITORIAL,Antioquia,Medellín
8,5,701427445,secretaria distrital gobierno,899999061,Territorial,No Definido,No Definido


In [23]:
tvec['NIT Entidad'] = tvec['NIT Entidad'].astype(str)
tvec['NIT Entidad'] = tvec['NIT Entidad'].str.replace('.0','')
nits_homologados['NIT'] = nits_homologados['NIT'].astype(str)
tvec_filtrado = tvec[tvec['NIT Entidad'].str.contains('|'.join(nits_homologados['NIT'].tolist()))]
tvec_filtrado.shape

C:\Users\Asus\AppData\Local\Temp\ipykernel_11512\3582684746.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nits_homologados['NIT'] = nits_homologados['NIT'].astype(str)


(328185, 20)

In [24]:
tvec_filtrado.head()

,Año,Identificador de la Orden,Agregacion,Rama de la Entidad,Sector de la Entidad,Entidad,Orden de la Entidad,NIT Entidad,Solicitante,Fecha,Proveedor,Estado,Solicitud,Items,Total,Ciudad,Entidad Obigada,EsPostconflicto,NIT proveedor,Actividad Economica Proveedor
12,2023,105565,Combustible Nacional,Ejecutiva,NaN,MUNICIPIO DE TULUA,Territorial,891900272,John Jairo Gomez Aguirre,02/28/2023 12:00:00 AM,Sodexo SAS,Emitida,180058.0,Combustible Categora C,70000000.0,TULUÁ - VALLE DEL CAUCA,No obligada,No Definido,800219876,8299
19,2023,108898,Material Pedaggico,Ejecutiva,Interior,CASANAREALCALDIA DE YOPAL,Territorial,891855017,Aydee Soler Sanabria,05/04/2023 12:00:00 AM,OFIX SAS,Emitida,186205.0,mpg01PA1125 PAPELERA 125 PAPEL BOND mpg01IVA,1638340.0,YOPAL,NaN,No Definido,9001568261,4690
20,2023,108898,Material Pedaggico,Ejecutiva,Interior,CASANAREALCALDIA DE YOPAL,Territorial,891855017,Aydee Soler Sanabria,05/04/2023 12:00:00 AM,OFIX SAS,Emitida,186205.0,mpg01PA1125 PAPELERA 125 PAPEL BOND mpg01IVA,1638340.0,YOPAL,NaN,No Definido,9001568261,4690
62,2023,103853,Combustible Nacional II,Ejecutiva,Funcion Publica,GOBERNACION DEL VICHADA,Territorial,800094067,Alvaro Arley Leon Florez,01/18/2023 12:00:00 AM,Sodexo SAS,Emitida,164478.0,Combustible Categora D,1.0,Puerto Carreño,No obligada,No Definido,800219876,8299
77,2023,119733,Grandes Superficies,Ejecutiva,Transporte,SERVICIO NACIONAL DE APRENDIZAJE SENA CENTRO ...,Nacional,899999034,Luis Prada Paez,11/14/2023 12:00:00 AM,CENCOSUD COLOMBIA SA,Cerrada,200660.0,GSF01KIT TALADRO PERCUTOR 12 PUL ACCES BLACK ...,4997650.0,girardot,Obligada,No Definido,900155107,4711


In [26]:
tvec_filtrado = tvec_filtrado[tvec_filtrado['Agregacion'].isin(['Servicio de Almacenamiento Ensamble y Distribucin de Refrigerios Escolares IV',
'Servicio de Recepcin Almacenamiento Ensamble y Distribucin de Refrigerios Escolares III',
'Dotacin Escolar II', 'Material Pedaggico', 'Suministro de alimentos PAE III', 'Suministro de alimentos PAE IV',
'Suministro de Alimentos PAE III'])]
tvec_filtrado.shape

(23374, 20)

In [29]:
tvec_filtrado.to_excel('../../muestras de datos/plata/educacion_tvec.xlsx', index=False)